# Assignment 4 for Course 1MS041
Make         sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline         and your highest score will be used.

---
## Assignment 4, PROBLEM 1
Maximum Points = 24


This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `mammography.mat` that you can download from [http://odds.cs.stonybrook.edu/mammography-dataset/](http://odds.cs.stonybrook.edu/mammography-dataset/). Below you can find the code to load this file into `X` and `Y`, you just need to put the file in your `data` folder. During mammography the doctor would be looking for something called `calcification`, which is calcium deposits in the breast tissue and is used as an indicator of cancer. In this dataset the `X` corresponds to some measurements, there are 6 features. The `Y` is a 0-1 label where 1 represents calcification and 0 does not.

1. [3p] Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, I have prepared a shuffling with a fixed seed, this way I can make sure that we all did the same splits. That is [train,test,validation] is the splitting layout.
2. [4p] Train a machine learning model on the training data (you are free to choose it yourself). Hint: you could always try `LogisticRegression`, but for it to work well you would need `StandardScaler` and put everything in a `Pipeline`.
3. [3p] Use the classification report from `Utils` and compute the intervals for precision-recall etc on the test set with `union_bound correction` with 95% confidence.
4. [3p] You are interested in minimizing the average cost of your classifier, but first we must define it:
    * If someone is calcified but classified as not, we say it costs 30 **(this is the worst scenario)** 
    * If someone is not calcified but classified as calcified, we say it costs 5 **(this probably only costs extra investigation)**
    * If someone is calcified but classified as calcified, costs 0 **(We did the right thing, no cost)**
    * If someone is not calcified but classified as not, costs 0 **(We did the right thing, no cost)**.

complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). What would be the cost of having a model that always classifies someone as not calcified on the test set?

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, we do that by checking say 100 evenly spaced proposal thresholds between 0 and 1, and use our testing data to compute the cost.
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 95% confidence.
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted, if we define the random variable
$$
    C = 30(1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
$$
then $C$ denotes the cost of a randomly chosen patient. In the above we are estimating $\mathbb{E}[C]$ using the empirical mean. However, since the number of calcifications in the population is fairly small and our classifier probably has a fairly high precision for the $0$ class, then $C$ should have fairly small variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

In [1]:
import scipy.io as so
import numpy as np
from sklearn.model_selection import train_test_split
data = so.loadmat('data/mammography.mat')

np.random.seed(0)
shuffle_index = np.arange(0,len(data['X']))
np.random.shuffle(shuffle_index)

X = data['X'][shuffle_index,:]
Y = data['y'][shuffle_index,:].flatten()

In [2]:

# Part 1

# Split the X,Y into three parts, make sure that the sizes are
# (6709, 6), (1677, 6), (2797, 6), (6709,), (1677,), (2797,)

X_train, X_te, Y_train, Y_te = train_test_split(X,Y, shuffle=False, test_size=0.4)
X_test, X_valid, Y_test, Y_valid = train_test_split(X_te, Y_te, shuffle=False, test_size=0.625)

In [3]:

# Part 2
# Use X_train,Y_train to train a model from sklearn. Make sure it has the predict_proba function
# for instance LogisticRegression
import sklearn.discriminant_analysis as skl_da
import sklearn.linear_model as skl_lm
from sklearn import tree

#model = skl_da.QuadraticDiscriminantAnalysis()
#model = skl_lm.LogisticRegression(solver='liblinear')
model = tree.DecisionTreeClassifier(max_depth=3)

model.fit(X_train, Y_train)

DecisionTreeClassifier(max_depth=3)

In [4]:
from sklearn.metrics import precision_recall_fscore_support

def precision_recall(y_true,
    y_pred,
    labels=None,alpha=0.01, correction=1):
    p = []
    r = []
    f1 = []
    support = []
    for label in labels:
        y_true_pred_label = y_true[y_pred == label]
        precision = np.mean(y_true_pred_label == label)
        delta = (1/np.sqrt(len(y_true_pred_label)))*np.sqrt((1/2)*np.log(2*correction/alpha))
        p.append("%.2f : [%.2f,%.2f]" % (precision, np.maximum(precision-delta,0),np.minimum(precision+delta,1)))
        
        y_pred_true_label = y_pred[y_true == label]
        recall = np.mean(y_pred_true_label == label)
        delta = (1/np.sqrt(len(y_pred_true_label)))*np.sqrt((1/2)*np.log(2*correction/alpha))
        r.append("%.2f : [%.2f,%.2f]" % (recall, np.maximum(recall-delta,0),np.minimum(recall+delta,1)))
        
    return (p,r)

def accuracy_interval(y_true,y_pred,alpha=0.01,correction=1):
    acc = np.mean(y_true == y_pred)
    delta = (1/np.sqrt(len(y_true)))*np.sqrt((1/2)*np.log(2*correction/alpha))
    return "%.2f : [%.2f,%.2f]" % (acc, np.maximum(acc-delta,0),np.minimum(acc+delta,1))

def classification_report_interval(
    y_true,
    y_pred,
    labels=None,
    alpha = 0.01,
    union_bound_correction=True
):
    digits = 18
    target_names = None
    if labels is None:
        labels = list(set(y_true).union(set(y_pred)))
        labels_given = False
    else:
        labels = np.asarray(labels)
        labels_given = True

    target_names = ["%s" % l for l in labels]

    headers = ["precision", "recall"]
    # compute per-class results without averaging
    # Simple correction using the union bound
    # We are computing 2 intervals for each label for precision and recall
    # In addition we are computing 2 intervals for accuracy
    # This is in total 2*n_labels+2
    if (union_bound_correction):
        correction = 2*len(labels)+2
    else:
        correction=1
    p, r = precision_recall(
        y_true,
        y_pred,
        labels=labels,
        alpha=alpha,
        correction=correction
    )

    rows = zip(target_names, p, r)

    name_width = max(len(cn) for cn in target_names)
    width = max(name_width, digits)
    head_fmt = "{:>{width}s} " + " {:>{digits}}" * len(headers)
    report = head_fmt.format("labels", *headers, width=width,digits=digits)
    report += "\n\n"
    row_fmt = "{:>{width}s} " + " {:>{digits}s}" * 2 + "\n"
    for row in rows:
        report += row_fmt.format(*row, width=width, digits=digits)
    row_fmt_acc = "{:>{width}s} " + " {:>{digits}s}" * 2 + " {:>{digits}s}""\n"
    report += "\n"
    accuracy = accuracy_interval(y_true,y_pred,alpha=alpha,correction=correction)
    report+=row_fmt_acc.format(*("accuracy","","",accuracy),width=width,digits=digits)

    return report


# Part 3

# Each precision and recall should be a tuple, for instance you can write
# precision0 = (0.9,0.95)

predictions = model.predict(X_test)

print(classification_report_interval(Y_test, predictions,union_bound_correction=True, alpha=0.05))

precision0 = (0.94,1.00)
recall0 = (0.96,1.00)
precision1 = (0.37, 1.00)
recall1 = (0.04,0.57)

# The code below will check that you supply the proper type
assert(type(precision0) == tuple)
assert(len(precision0) == 2)
assert(type(recall0) == tuple)
assert(len(recall0) == 2)
assert(type(precision1) == tuple)
assert(len(precision1) == 2)
assert(type(recall1) == tuple)
assert(len(recall1) == 2)

            labels           precision             recall

                 0  0.98 : [0.94,1.00] 1.00 : [0.96,1.00]
                 1  0.80 : [0.37,1.00] 0.31 : [0.04,0.57]

          accuracy                                        0.98 : [0.94,1.00]



In [5]:

# Part 4

def cost(model,threshold,X,Y):
    pred_proba = model.predict_proba(X)[:,1]
    predictions = (pred_proba >= threshold)*1
    
    cost = 0
    
    for i in range(len(Y)):
        if (predictions[i] == Y[i]):
            cost = cost + 0
        elif (predictions[i] == 0):
            cost = cost + 30
        elif (predictions[i] == 1):
            cost = cost + 5
            
    cost = cost/len(Y)
    
    # Fill in what is missing to compute the cost and return it
    # Note that we are interested in average cost (cost per person)
    
    return cost

In [6]:

# Part 4

# Fill in the naive cost of the model that would classify all as non-calcified on the test set

naive_cost = cost(model, 1, X_test, Y_test)
naive_cost

0.6976744186046512

In [7]:
import math
# Part 5

thresholds = np.linspace(0,1,num=100)

optimal_threshold = math.inf
cost_at_optimal_threshold = math.inf

for i in range(len(thresholds)):
    cost_val = cost(model, thresholds[i], X_test, Y_test)
    if (cost_val < cost_at_optimal_threshold):
        cost_at_optimal_threshold = cost_val
        optimal_threshold = thresholds[i]
        
print(optimal_threshold)
print(cost_at_optimal_threshold)

0.15151515151515152
0.4502087060226595


In [8]:

# Part 6

cost_at_optimal_threshold_validation = cost(model, 0.15151515151515152 , X_valid , Y_valid)

print(cost_at_optimal_threshold_validation)

cost_at_optimal_threshold_validation_2 = cost(model, 0.15151515151515152 , X_te , Y_te)
print(cost_at_optimal_threshold_validation_2)

alpha = 0.05
delta = (1/np.sqrt(len(Y_test)))*np.sqrt((1/2)*np.log(2/alpha))
print(delta)

 
# Report the cost interval as a tuple cost_interval = (a,b)

cost_interval = (0.00,1.12)
# The code below will tell you if you filled in the intervals correctly
assert(type(cost_interval) == tuple)
assert(len(cost_interval) == 2)

0.3503754022166607
0.3877961555654895
0.0331639080701232


In [9]:
def cost(model,threshold,X,Y):
    pred_proba = model.predict_proba(X)[:,1]
    predictions = (pred_proba >= threshold)*1
    
    cost = 0
    validation_mean = 0.3503754022166607
    var =0
    
    for i in range(len(Y)):
        if (predictions[i] == Y[i]):
            cost = cost + 0
            var = var + math.pow(validation_mean - 0, 2)
        elif (predictions[i] == 0):
            cost = cost + 30
            var = var + math.pow(validation_mean - 30, 2)
        elif (predictions[i] == 1):
            cost = cost + 5
            var = var + math.pow(validation_mean - 5, 2)
            
    cost = cost/len(Y)
    var = var/len(Y)
    
    print(var)
    
    # Fill in what is missing to compute the cost and return it
    # Note that we are interested in average cost (cost per person)
    
    return cost
# Part 7

cost(model, 0.15151515151515152 , X_valid , Y_valid)

variance_of_C = 9.941591743234143

9.941591743234143


In [10]:
import sys
sys.path.append("..")

from Utils import epsilon_bounded, print_confidence_interval, bennett_epsilon

# Part 7

b = 30 - 0
sigma=np.sqrt(variance_of_C)

eb = epsilon_bounded(len(Y_valid), b, 0.05)
be = bennett_epsilon(len(Y_valid), b, sigma, 0.05)

# A 95% confidence interval of the random variable C using Bennett's inequality
# epsilon = bennett_epsilon(len(Y_valid), 5, variance_of_C, 0.05)
# interval_of_C = get_confidence_interval(len(Y_valid), epsilon, 0.05)

interval_of_C = (0.3503754022166607 - be, 0.3503754022166607 + be)

print(interval_of_C)


assert(type(interval_of_C) == tuple)
assert(len(interval_of_C) == 2)

Numerical error -1.0651202142497596e-15
(0.1757278095265725, 0.5250229949067489)
